In [ ]:
#downloading the dataset from github
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [1]:
#reading the input files....

with open('input.txt','r',encoding='utf-8') as f :
    text = f.read()
    

In [2]:
print(len(text))

1115394


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
#creating the vocabulary of the dataset with identifying the number of unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
#tokenization

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s : [stoi[c] for c in s]
decode = lambda l : ''.join([itos[c] for c in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
#encoding the dataset into a torch.tensor with defined tokenizer above
import torch

data = torch.tensor(encode(text),dtype=torch.long)


In [8]:
print(data.shape)

print(data[:10])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [9]:
#train and validation dataset
n = int(.9*len(text))
train_data = data[:n]
val_data   = data[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target  = y[t]
    print(f"When input is {context} the target : {target}")

When input is tensor([18]) the target : 47
When input is tensor([18, 47]) the target : 56
When input is tensor([18, 47, 56]) the target : 57
When input is tensor([18, 47, 56, 57]) the target : 58
When input is tensor([18, 47, 56, 57, 58]) the target : 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target : 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target : 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target : 58


In [14]:
torch.randint(len(data) - block_size,(batch_size,))

tensor([1078327,  453969,   41646,  671252])

In [18]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    x  = torch.stack([data[i:i+block_size] for i in ix])
    y  = torch.stack([data[i+1:i+block_size+1] for i  in ix])
    return x,y

xb,yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)

print('targets')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target  = yb[b,t]
        print(f"When input is {context} the target : {target}")

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
When input is tensor([24]) the target : 43
When input is tensor([24, 43]) the target : 58
When input is tensor([24, 43, 58]) the target : 5
When input is tensor([24, 43, 58,  5]) the target : 57
When input is tensor([24, 43, 58,  5, 57]) the target : 1
When input is tensor([24, 43, 58,  5, 57,  1]) the target : 46
When input is tensor([24, 43, 58,  5, 57,  1, 46]) the target : 43
When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target : 39
When input is tensor([44]) the target : 53
When input is tensor([44, 53]) the target : 56
When input is tensor([44, 53, 56]) the target : 1
When input is t

In [104]:
#bi-gram language model....
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLM(nn.Module):
    
    def __init__(self,vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    def forward(self,idx,targets=None):
        
        #logits and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) #outputs of size (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        
        return logits,loss
    
    def generate(self,idx,max_tokens):
        #idx is the input in (B,T) shape in current context
        
        for _ in range(max_tokens):
            #get the prediction
            logits,loss = self(idx)
            #focus only on the last time step
            logits = logits[:,-1,:] #becomes (B,C)
            #apply softmax to get the probabilities
            probs = F.softmax(logits,dim=1)
            #sample from the distribution
            idx_nxt = torch.multinomial(probs,num_samples=1) #(B,1)
            #append sampled index to the running sequence
            idx = torch.cat((idx,idx_nxt),dim=1) #(B,T+1)
        return idx

m = BigramLM(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape,loss)

torch.Size([32, 65]) tensor(4.8786, grad_fn=<NllLossBackward0>)


In [113]:
predictions= m.generate(torch.zeros((1,1),dtype=torch.long),100)

In [114]:
decode(kk[0].tolist())

"\nr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3!"

In [115]:
#pytorch optimizer....
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [119]:
batch_size = 32

for steps in range(10000):
    #sample the batch of data
    xb,yb = get_batch('train')
    
    #eval loss
    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.4192638397216797


In [122]:
predictions= decode(m.generate(torch.zeros((1,1),dtype=torch.long),100)[0].tolist())

print(predictions)


Toustoigpat ICardef ld s's; I wove a aveny card m th ffegiANCord
Sewing.
Y:
Nucor herend's mio d ben


## Trick in self-attention

In [123]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [124]:
#We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))

for b in range(B):
    for t in range(T):
        x_prev = x[b,:t+1]
        xbow[b,t] = torch.mean(x_prev,0)

In [126]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [127]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [132]:
wei = torch.tril(torch.ones((T,T)))
wei = wei/wei.sum(1,keepdim=True)
xbow2 = wei@x # (B,T,T)x (B,T,C) ----> (B,T,C)
torch.allclose(xbow,xbow2)

True

In [133]:
#using softmax
tril = torch.tril(torch.ones((T,T)))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=1)
xbow3 = wei@x

torch.allclose(xbow,xbow3)

True

In [1]:
import torch
from torch.nn import functional as F
import torch.nn as nn
#using softmax
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)
print(x.shape)
head_size = 16 

key = nn.Linear(C,head_size,bias=False) # (B,T,16)
query = nn.Linear(C,head_size,bias=False) # (B,T,16)
value = nn.Linear(C,head_size,bias=False)
k = key(x)
q = query(x)


wei = q @ k.transpose(-2,-1) # (B,T,16) @ (B,16,T) ---> (B,T,T)

tril = torch.tril(torch.ones((T,T)))
wei = wei.masked_fill(tril==0,float('-inf'))
wei = F.softmax(wei,dim=-1)


v = value(x)
out = wei@v
out.shape

torch.Size([4, 8, 32])


torch.Size([4, 8, 16])

In [131]:
torch.manual_seed(42)
a = torch.tril(torch.ones((3,3)))
a = a/torch.sum(a,1,keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a@b

print(a)

print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C,head_size,bias=False)
query = nn.Linear(C,head_size,bias=False)


k = key(x)
q = query(x)

w = q @ k.transpose(-2,-1)

tril = torch.tril(torch.ones((T,T)))

w = w.masked_fill(tril==0,float('-inf'))
print(w[0])

w = F.softmax(w,dim=-1)